In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
# Load Data Train csv
train = pickle.load(open('Train_Test/train_bams.p','rb'))
test = pickle.load(open('Train_Test/test_bams.p','rb'))

In [3]:
train

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
125,0,0.0382,0.001459,138.1194,72.933239,17,18,59,36,23,...,0.174261,0.530126,0.512059,0.408730,31.318280,174.085678,385.225921,0.674239,1.450915,5.958
74,1,-0.7970,0.635209,77.2375,41.352102,6,6,37,23,14,...,0.101150,0.592622,0.550249,0.332241,18.177672,43.726934,87.066620,0.775200,1.475112,5.854
1,0,-0.0693,0.004802,113.5328,61.298032,0,0,54,30,24,...,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973,6.292
110,0,-2.5774,6.642991,128.7353,70.286825,17,18,60,35,25,...,0.245975,0.488948,0.408430,0.497519,24.554292,143.408571,358.730867,0.514894,1.394896,7.921
99,1,-0.4080,0.166464,117.9068,62.001653,12,12,54,33,21,...,0.325881,0.553109,0.498577,0.513381,21.955191,131.000413,332.588534,0.377649,1.565066,5.854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0,-2.0910,4.372281,119.9176,64.300032,12,12,57,33,24,...,0.119319,0.611371,0.550971,0.478145,25.754579,105.897604,242.885706,0.720412,1.640487,6.180
95,0,-0.1589,0.025249,142.5628,78.271583,12,12,70,39,31,...,0.153062,0.639028,0.396740,0.351866,39.688457,278.796848,771.668789,0.680110,1.387634,5.585
32,0,-0.4082,0.166627,106.0673,58.637653,12,12,51,30,21,...,0.344943,0.576989,0.426777,0.404989,17.907066,96.524233,263.097835,0.277946,1.408755,8.699
124,0,-0.8291,0.687407,140.0467,75.315618,17,18,63,37,26,...,0.166500,0.557440,0.485636,0.409972,31.807655,174.995325,389.991476,0.685353,1.453048,7.229


In [4]:
# Load Pickle 
label_25 = pickle.load(open("desc/new_desc25.pkl", "rb"))
label_25

['C2SP3',
 'maxssCH2',
 'AATSC6e',
 'RDF30u',
 'MATS2i',
 'RDF30e',
 'ATSC6m',
 'SpMax3_Bhs',
 'SRW5',
 'MDEC-34',
 'minHBint4',
 'TDB4i',
 'nssCH2',
 'SHCsats',
 'SHBint4',
 'SC-4',
 'StN',
 'ntsC',
 'maxtsC',
 'ATSC4p',
 'ATSC4p',
 'SsF',
 'SC-6',
 'MDEC-24',
 'LipoaffinityIndex']

In [5]:
x_train = train.loc[:,label_25]
x_test = test.loc[:,label_25]
y_train = train.iloc[:,[-1]]
y_test = test.iloc[:,[-1]]

In [6]:
x_test.shape

(27, 25)

In [7]:
mmscaler = MinMaxScaler()
mmscaler.fit(x_train)
sc_x_train = mmscaler.transform(x_train)
sc_x_test = mmscaler.transform(x_test)

In [8]:
paramgrid = {
    'C' : [0.1,1,10,100,1000],
    'gamma' : ['auto','scale'],
    'epsilon' : [0.1,1,10,100,1000]
}

In [9]:
grid = GridSearchCV(
        estimator = SVR(kernel = 'rbf'),
        param_grid = paramgrid,
        cv = 5,
        scoring = 'neg_mean_squared_error',
        n_jobs = -1
)
grid.fit(sc_x_train, y_train)

C:\Users\Bams\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'epsilon': [0.1, 1, 10, 100, 1000],
                         'gamma': ['auto', 'scale']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [10]:
grid.best_params_

{'C': 10, 'epsilon': 0.1, 'gamma': 'scale'}

In [11]:
pickle.dump(grid.best_params_, open( "data_RBF/new_params_rbf_25.p", "wb" ))

In [12]:
# model
model = SVR(kernel = 'rbf', C = grid.best_params_['C'],
           gamma = grid.best_params_['gamma'],
           epsilon = grid.best_params_['epsilon'])
model.fit(sc_x_train, y_train)

C:\Users\Bams\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [13]:
y_train_pred = model.predict(sc_x_train)
y_test_pred = model.predict(sc_x_test)

In [14]:
y_test_pred

array([5.66999603, 5.34043776, 5.75334376, 6.26503724, 7.65427444,
       7.11119598, 8.21360988, 5.70800374, 6.90282228, 5.55621083,
       5.51852904, 8.349151  , 8.66278684, 6.78191299, 5.53641938,
       5.46922363, 8.79378654, 5.41899973, 6.62649621, 7.62191279,
       6.63753611, 8.48119089, 7.03775316, 7.06929543, 8.53860792,
       8.33502183, 5.73470512])

In [15]:
pickle.dump(y_test_pred, open( "data_RBF/y_test_pred_rbf25.p", "wb" ))

In [16]:
y_train_pred

array([7.16794205, 5.75410207, 6.34830492, 7.80745824, 5.5569618 ,
       8.53757103, 7.20076971, 8.55782597, 5.61886951, 6.97469935,
       7.46894575, 6.64093472, 8.16818767, 6.58047056, 8.12794109,
       5.86305989, 6.92039459, 5.55536344, 5.53219138, 8.41923388,
       8.10061902, 8.85879322, 7.31055364, 6.68535774, 8.78604101,
       6.3812185 , 5.66948597, 5.92426521, 8.72385105, 5.89650412,
       6.53983788, 5.62313827, 5.58079573, 5.23359846, 6.45541858,
       5.60918671, 6.47710313, 6.1693531 , 6.63461835, 5.69574907,
       5.57673571, 5.69587907, 6.82062584, 5.76502177, 6.21040945,
       6.02097958, 5.78323317, 5.79887732, 5.62286198, 6.45856542,
       5.79947344, 5.84511861, 6.26676944, 5.1509264 , 6.82089022,
       6.89578749, 8.10083412, 7.25983737, 5.14075654, 7.02588369,
       6.62562983, 6.33841591, 5.32385322, 6.32790999, 5.71647617,
       5.67074717, 8.15177594, 6.65268152, 6.33124243, 8.32875779,
       6.05386655, 6.40893539, 5.82127249, 6.05537585, 6.80909

In [17]:
pickle.dump(y_train_pred, open( "data_RBF/y_train_pred_rbf25.p", "wb" ))

In [18]:
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [19]:
train_r2

0.9597826530311843

In [20]:
pickle.dump(train_r2, open( "data_RBF/new_train_r2_rbf25.p", "wb" ))

In [21]:
test_r2

0.8444372623187069

In [22]:
pickle.dump(test_r2, open( "data_RBF/new_test_r2_rbf25.p", "wb" ))

In [23]:
# # Load Pickle 
params5 = pickle.load(open("data_RBF/new_params_rbf_5.p", "rb"))
train_rbf5 = pickle.load(open("data_RBF/new_train_r2_rbf5.p", "rb"))
test_rbf5 = pickle.load(open("data_RBF/new_test_r2_rbf5.p", "rb"))
test_pred_rbf5 = pickle.load(open("data_RBF/y_test_pred_rbf5.p", "rb"))
train_pred_rbf5 = pickle.load(open("data_RBF/y_train_pred_rbf5.p", "rb"))

params10 = pickle.load(open("data_RBF/new_params_rbf_10.p", "rb"))
train_rbf10 = pickle.load(open("data_RBF/new_train_r2_rbf10.p", "rb"))
test_rbf10 = pickle.load(open("data_RBF/new_test_r2_rbf10.p", "rb"))
test_pred_rbf10 = pickle.load(open("data_RBF/y_test_pred_rbf10.p", "rb"))
train_pred_rbf10 = pickle.load(open("data_RBF/y_train_pred_rbf10.p", "rb"))

params15 = pickle.load(open("data_RBF/new_params_rbf_15.p", "rb"))
train_rbf15 = pickle.load(open("data_RBF/new_train_r2_rbf15.p", "rb"))
test_rbf15 = pickle.load(open("data_RBF/new_test_r2_rbf15.p", "rb"))
test_pred_rbf15 = pickle.load(open("data_RBF/y_test_pred_rbf15.p", "rb"))
train_pred_rbf15 = pickle.load(open("data_RBF/y_train_pred_rbf15.p", "rb"))

params20 = pickle.load(open("data_RBF/new_params_rbf_20.p", "rb"))
train_rbf20 = pickle.load(open("data_RBF/new_train_r2_rbf20.p", "rb"))
test_rbf20 = pickle.load(open("data_RBF/new_test_r2_rbf20.p", "rb"))
test_pred_rbf20 = pickle.load(open("data_RBF/y_test_pred_rbf20.p", "rb"))
train_pred_rbf20 = pickle.load(open("data_RBF/y_train_pred_rbf20.p", "rb"))

params25 = pickle.load(open("data_RBF/new_params_rbf_25.p", "rb"))
train_rbf25 = pickle.load(open("data_RBF/new_train_r2_rbf25.p", "rb"))
test_rbf25 = pickle.load(open("data_RBF/new_test_r2_rbf25.p", "rb"))
test_pred_rbf25 = pickle.load(open("data_RBF/y_test_pred_rbf25.p", "rb"))
train_pred_rbf25 = pickle.load(open("data_RBF/y_train_pred_rbf25.p", "rb"))

In [ ]:
mse_train5 = (mean_squared_error)

In [24]:
params5, train_rbf5, test_rbf5, test_pred_rbf5, train_pred_rbf5

({'C': 10, 'epsilon': 0.1, 'gamma': 'scale'},
 0.8857537956041112,
 0.7335067689924518,
 array([5.90585335, 5.55728622, 5.6409498 , 6.24215527, 6.85641501,
        6.18171158, 7.87376709, 5.61779912, 6.85338412, 5.60832289,
        5.72724202, 8.53125618, 8.71781747, 6.46867144, 5.27119913,
        5.17737639, 7.07208831, 5.90435786, 6.98675459, 6.82616707,
        6.34776157, 8.25346627, 6.78368562, 6.25444034, 8.48143077,
        8.53533441, 5.71283114]),
 array([7.08241069, 5.75380128, 5.77349536, 6.57679143, 5.65676281,
        8.31772569, 6.50210502, 8.41582438, 5.58969783, 6.67285106,
        8.14360121, 6.64113484, 8.2153278 , 5.96810493, 8.06706976,
        5.76352911, 7.09980354, 5.56877467, 5.34166622, 8.49031367,
        7.9738946 , 8.85920121, 6.97756123, 6.49276054, 8.84398947,
        6.4873202 , 5.58969783, 6.78781717, 8.72402953, 6.42038319,
        6.72107477, 5.49310202, 5.7603176 , 5.57761659, 6.29666569,
        6.108422  , 6.92386348, 5.97760455, 6.31472523, 5.7889

In [25]:
params10, train_rbf10, test_rbf10, test_pred_rbf10, train_pred_rbf10

({'C': 10, 'epsilon': 0.1, 'gamma': 'scale'},
 0.9230656762462885,
 0.8325258077641262,
 array([5.68593123, 5.5821817 , 5.94806754, 6.38643404, 7.27925061,
        6.7131554 , 7.70196956, 5.82750928, 6.65440835, 5.71395663,
        5.61196247, 8.5418063 , 8.59395405, 6.33563501, 5.93060377,
        5.85084762, 8.72493196, 5.68502224, 6.64423249, 7.61416665,
        6.59196798, 8.45088782, 6.99674641, 7.51343094, 8.51573942,
        8.6305712 , 5.87574774]),
 array([6.60136833, 5.75383492, 6.39149035, 7.71275272, 5.66141985,
        8.53836519, 7.20139268, 7.97774262, 5.66895993, 6.75134353,
        7.46857219, 6.6408535 , 6.90938779, 6.47245355, 8.26670113,
        5.71031917, 7.04884825, 5.56854677, 5.5051184 , 8.41920807,
        8.30119906, 8.67593369, 7.48333402, 7.09951009, 8.78566282,
        6.23068559, 5.66895993, 6.17274389, 8.72433978, 6.27892877,
        6.72070967, 5.55290636, 5.74537057, 5.41456343, 6.49709848,
        6.0164352 , 6.49685121, 5.8852434 , 6.64523514, 5.5238

In [26]:
params15, train_rbf15, test_rbf15, test_pred_rbf15, train_pred_rbf15

({'C': 10, 'epsilon': 0.1, 'gamma': 'scale'},
 0.9227380860537745,
 0.7949840116771972,
 array([5.74878241, 5.50432883, 6.01341531, 6.06626199, 7.73253551,
        6.57837906, 7.5528432 , 5.75076294, 6.85376029, 5.449382  ,
        5.64422221, 8.4146105 , 8.64869414, 6.37291046, 5.66457775,
        5.3873514 , 8.47358266, 5.59778322, 5.80679054, 6.92921988,
        6.71744716, 8.31908091, 6.89630692, 5.8712037 , 8.58248384,
        8.28936791, 5.71962104]),
 array([6.77705709, 5.75388626, 6.19201999, 7.09958091, 5.54288226,
        8.57266371, 6.48081852, 8.55801634, 5.75815441, 7.16590598,
        7.4151356 , 6.64125496, 8.16849365, 6.55320261, 8.06687875,
        5.71927005, 6.89979447, 5.89738112, 5.5173095 , 8.21886449,
        8.10084557, 8.85884599, 7.12648341, 6.5489836 , 8.78565553,
        6.48673799, 5.6794776 , 5.92391547, 8.72402443, 5.89612487,
        6.72081908, 5.67316945, 5.58090592, 5.32881023, 6.49731743,
        6.2121966 , 6.81055654, 5.96901352, 6.58381972, 5.4861

In [27]:
params20, train_rbf20, test_rbf20, test_pred_rbf20, train_pred_rbf20

({'C': 1000, 'epsilon': 1, 'gamma': 'auto'},
 0.6734696928821622,
 0.5331861128038671,
 array([5.76355074, 5.71865246, 6.2118491 , 5.63846294, 8.27818026,
        7.0637909 , 8.20150658, 5.66093043, 7.15400337, 5.66355443,
        5.7390987 , 7.75315491, 7.72040736, 6.53130058, 5.85638949,
        5.52567946, 8.08872319, 6.58470215, 4.78733839, 6.91479287,
        6.75472959, 7.22662777, 6.57484476, 5.0954482 , 7.67650089,
        7.44373215, 6.09148249]),
 array([6.95838782, 5.75553795, 5.78752558, 6.92105486, 5.84000794,
        7.65869466, 6.30096781, 7.65779656, 6.22968672, 6.7666748 ,
        6.56852851, 6.32554748, 7.40943107, 6.07917739, 7.30093148,
        6.03352677, 6.8504405 , 5.89528364, 6.39483665, 8.13397022,
        7.25219244, 7.95897093, 7.30205206, 7.14640655, 8.36542272,
        6.34682662, 6.22968672, 6.82409856, 7.8243505 , 6.79588352,
        6.32920127, 5.86265197, 5.62465207, 5.77285109, 6.97271158,
        6.13495054, 6.48751285, 6.29988326, 5.89385735, 5.62492

In [28]:
params25, train_rbf25, test_rbf25, test_pred_rbf25, train_pred_rbf25

({'C': 10, 'epsilon': 0.1, 'gamma': 'scale'},
 0.9597826530311843,
 0.8444372623187069,
 array([5.66999603, 5.34043776, 5.75334376, 6.26503724, 7.65427444,
        7.11119598, 8.21360988, 5.70800374, 6.90282228, 5.55621083,
        5.51852904, 8.349151  , 8.66278684, 6.78191299, 5.53641938,
        5.46922363, 8.79378654, 5.41899973, 6.62649621, 7.62191279,
        6.63753611, 8.48119089, 7.03775316, 7.06929543, 8.53860792,
        8.33502183, 5.73470512]),
 array([7.16794205, 5.75410207, 6.34830492, 7.80745824, 5.5569618 ,
        8.53757103, 7.20076971, 8.55782597, 5.61886951, 6.97469935,
        7.46894575, 6.64093472, 8.16818767, 6.58047056, 8.12794109,
        5.86305989, 6.92039459, 5.55536344, 5.53219138, 8.41923388,
        8.10061902, 8.85879322, 7.31055364, 6.68535774, 8.78604101,
        6.3812185 , 5.66948597, 5.92426521, 8.72385105, 5.89650412,
        6.53983788, 5.62313827, 5.58079573, 5.23359846, 6.45541858,
        5.60918671, 6.47710313, 6.1693531 , 6.63461835, 5.6957